# Acid
```
'RM01/0001'
'RM01/0004'
'RM01/0006'
'RM01/0007'
```

In [2]:
from src import extract_and_clean as ec
from src import transform as ts
from src import feature_engineer as fe
import src.forecastor as fc
import numpy as np

# Preparations
# Import data
gas_df = ec.get_Fred_data('PNGASEUUSDM',2014,2024)
wheat_df = ec.get_Fred_data('PWHEAMTUSDM',2014,2024)
ammonia_df = ec.get_Fred_data('WPU0652013A',2014,2024)
elec_df = ec.clean_elec_csv('/Users/barryhuang/Projects/Raw_Material_Price_Prediction/data/raw/ELECTRICITY_03_2024.csv',2014,2024)

df = ec.clean_pred_price_evo_csv("/Users/barryhuang/Projects/Raw_Material_Price_Prediction/data/raw/Dataset_Future_Predicting_Price_Evolutions_202403.csv",2014,2023)

target = 'acid'.lower()

RM_codes = ['RM01/0001','RM01/0004','RM01/0006','RM01/0007']

external_drivers = {
    "PNGASEUUSDM": gas_df,
    "PWHEAMTUSDM": wheat_df,
    "WPU0652013A": ammonia_df,
    "Electricity": elec_df
}

test_periods = [
    ('2019-01-01', '2019-07-01'),
    ('2019-07-01', '2020-01-01'),
    ('2020-01-01', '2020-07-01'),
    ('2020-07-01', '2021-01-01'),
    ('2021-01-01', '2021-07-01'),
    ('2021-07-01', '2022-01-01'),
    ('2022-01-01', '2022-07-01'),
    ('2022-07-01', '2023-01-01'),
    ('2023-01-01', '2023-07-01'),
    ('2023-07-01', '2024-01-01')
]

lags = [1,3,6]

alpha_bottom = 0.01


In [ ]:
# Impute raw data of target variables 
imputed_df, missing = ts.impute_pred_price_evo_csv(df)

# Feature engineering
dummy_df = ts.get_dummies_and_average_price(imputed_df,target,*RM_codes)
feature_df = fe.generate_features(1,12,dummy_df,missing,*RM_codes, **external_drivers)
#
# if type(feature_df.Time) != "datetime64":
#     feature_df['Time'] = pd.to_datetime(feature_df['Time'])
#
# assert feature_df['Time'].dtype == "datetime64[ns]" , "df[Time] is not dataetime64."
#
# feature_df = feature_df[feature_df.Year >= 2016]

In [4]:
# Persistent Naive
for code in RM_codes:
    for lag in lags:
        mape_values = list()
        for period in test_periods:
            result = fc.persistence_Naive_MAPE(feature_df,code,lag,period)
            mape_values.append(result)
            
        assert len(mape_values) == len(test_periods), "len(mape_values)!=len(test_periods)"
        average_mape = np.mean(mape_values) 
        print(f"{target} {code}, {lag}-month lag, Naive, average MAPE: {average_mape:.3f}")

acid RM01/0001, 1-month lag, Naive, average MAPE: 4.317
acid RM01/0001, 3-month lag, Naive, average MAPE: 11.371
acid RM01/0001, 6-month lag, Naive, average MAPE: 21.451
acid RM01/0004, 1-month lag, Naive, average MAPE: 9.444
acid RM01/0004, 3-month lag, Naive, average MAPE: 14.865
acid RM01/0004, 6-month lag, Naive, average MAPE: 25.072
acid RM01/0006, 1-month lag, Naive, average MAPE: 9.801
acid RM01/0006, 3-month lag, Naive, average MAPE: 13.123
acid RM01/0006, 6-month lag, Naive, average MAPE: 21.071
acid RM01/0007, 1-month lag, Naive, average MAPE: 12.455
acid RM01/0007, 3-month lag, Naive, average MAPE: 16.383
acid RM01/0007, 6-month lag, Naive, average MAPE: 23.324


In [5]:
# Lasso with autoregression features only
for code in RM_codes:
    for lag in lags:
        mape_values = list()
        for period in test_periods:
            result = fc.train_model_AR(feature_df,code,lag,period,alpha_bottom)
            mape_values.append(result)
        
        assert len(mape_values) == len(test_periods), "len(mape_values)!=len(test_periods)"
        average_mape = np.mean(mape_values) 
        print(f"{target} {code}, {lag}-month lag, AR, average MAPE: {average_mape:.3f}")

acid RM01/0001, 1-month lag, AR, average MAPE: 5.536
acid RM01/0001, 3-month lag, AR, average MAPE: 12.693
acid RM01/0001, 6-month lag, AR, average MAPE: 26.898
acid RM01/0004, 1-month lag, AR, average MAPE: 9.663
acid RM01/0004, 3-month lag, AR, average MAPE: 14.158
acid RM01/0004, 6-month lag, AR, average MAPE: 22.682
acid RM01/0006, 1-month lag, AR, average MAPE: 9.289
acid RM01/0006, 3-month lag, AR, average MAPE: 13.792
acid RM01/0006, 6-month lag, AR, average MAPE: 21.592
acid RM01/0007, 1-month lag, AR, average MAPE: 16.910


/Users/barryhuang/Projects/Raw_Material_Price_Prediction/.venv/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.639e-02, tolerance: 1.455e-03
  model = cd_fast.enet_coordinate_descent(
/Users/barryhuang/Projects/Raw_Material_Price_Prediction/.venv/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.029e-02, tolerance: 1.455e-03
  model = cd_fast.enet_coordinate_descent(
/Users/barryhuang/Projects/Raw_Material_Price_Prediction/.venv/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might

acid RM01/0007, 3-month lag, AR, average MAPE: 24.730


/Users/barryhuang/Projects/Raw_Material_Price_Prediction/.venv/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.309e-03, tolerance: 1.455e-03
  model = cd_fast.enet_coordinate_descent(
/Users/barryhuang/Projects/Raw_Material_Price_Prediction/.venv/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.478e-02, tolerance: 1.455e-03
  model = cd_fast.enet_coordinate_descent(
/Users/barryhuang/Projects/Raw_Material_Price_Prediction/.venv/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might

acid RM01/0007, 6-month lag, AR, average MAPE: 28.414


In [6]:
# Lasso with autoregression features and external price drivers
for code in RM_codes:
    for lag in lags:
        mape_values = list()
        for period in test_periods:
            result = fc.train_model_all_features(feature_df,code,lag,period,alpha_bottom)
            mape_values.append(result)
        
        assert len(mape_values) == len(test_periods), "len(mape_values)!=len(test_periods)"
        average_mape = np.mean(mape_values) 
        print(f"{target} {code}, {lag}-month lag, all features, average MAPE: {average_mape:.3f}")

acid RM01/0001, 1-month lag, all features, average MAPE: 5.738
acid RM01/0001, 3-month lag, all features, average MAPE: 11.114
acid RM01/0001, 6-month lag, all features, average MAPE: 21.339
acid RM01/0004, 1-month lag, all features, average MAPE: 11.786
acid RM01/0004, 3-month lag, all features, average MAPE: 22.362
acid RM01/0004, 6-month lag, all features, average MAPE: 33.225
acid RM01/0006, 1-month lag, all features, average MAPE: 13.974
acid RM01/0006, 3-month lag, all features, average MAPE: 15.762
acid RM01/0006, 6-month lag, all features, average MAPE: 22.418


/Users/barryhuang/Projects/Raw_Material_Price_Prediction/.venv/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.470e-03, tolerance: 1.455e-03
  model = cd_fast.enet_coordinate_descent(
/Users/barryhuang/Projects/Raw_Material_Price_Prediction/.venv/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.303e-02, tolerance: 3.171e-03
  model = cd_fast.enet_coordinate_descent(


acid RM01/0007, 1-month lag, all features, average MAPE: 23.898


/Users/barryhuang/Projects/Raw_Material_Price_Prediction/.venv/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.146e-03, tolerance: 1.309e-03
  model = cd_fast.enet_coordinate_descent(
/Users/barryhuang/Projects/Raw_Material_Price_Prediction/.venv/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.678e-03, tolerance: 1.235e-03
  model = cd_fast.enet_coordinate_descent(
/Users/barryhuang/Projects/Raw_Material_Price_Prediction/.venv/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might

acid RM01/0007, 3-month lag, all features, average MAPE: 27.501


/Users/barryhuang/Projects/Raw_Material_Price_Prediction/.venv/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.508e-03, tolerance: 1.309e-03
  model = cd_fast.enet_coordinate_descent(
/Users/barryhuang/Projects/Raw_Material_Price_Prediction/.venv/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.518e-02, tolerance: 1.671e-03
  model = cd_fast.enet_coordinate_descent(


acid RM01/0007, 6-month lag, all features, average MAPE: 28.915
